In [3]:
# download nltk stopwords
import nltk
nltk.download('stopwords')

# Install a particular version of `google-cloud-storage` because (oddly enough) 
# the  version on Colab and GCP is old. A dependency error below is okay.
!pip install -q google-cloud-storage==1.43.0

# authenticate below for Google Storage access as needed
from google.colab import auth
auth.authenticate_user()
from google.colab import drive
drive.mount('/content/drive/')
import sys
sys.path.append('/content/drive/MyDrive/Information Retrieval/Project/information-retrieval-final-project/colab_cache')
%cd /content/drive/MyDrive/Information Retrieval/Project/information-retrieval-final-project/colab_cache

# install ngrok to emulate public IP / address
!wget -N https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip

# TODO: sign up for an ngrok account
# then put your ngrok token below, uncomment, and execute
!./ngrok authtoken 2K5pCuAwvo0Kajtz8DA5HwCRr7f_2Rssap33tUn3qcHuxFDBY

# install a ngrok python package and a version of flask that works with it in 
# colab
!pip -q install flask-ngrok
!pip -q install flask==0.12.2
# !pip -q install flask_restful

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 KB 3.5 MB/s eta 0:00:00
Mounted at /content/drive/
/content/drive/MyDrive/Information Retrieval/Project/information-retrieval-final-project/colab_cache
for details.

--2023-01-11 22:23:56--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 18.205.222.128, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  4.79MB/s    in 2.8s    

2023-01-11 22:24:00 (4.79 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
/bin/bash: ./ngrok: Permission denied
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 KB 3.1 MB/s eta 0:00:00


# Run the app

In [4]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 19.3 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=5922305cc9cb0c342bcd210f23bafae9da8d24675e46c74cc8f4deaac400df8a
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [18]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import numpy as np
from collections import defaultdict, Counter
import pandas as pd
from inverted_index_colab import *
from pyspark.sql import SparkSession
import colab_search_frontend as se
import time
import importlib
importlib.reload(se)

spark = SparkSession.builder.appName("colab_front").getOrCreate()
id_title_pagerank_pageviews = spark.read.format("csv").option("compression", "gzip").load("colab_pagerank.csv.gz",header = True).rdd.map(lambda x: (x.id, [x.title, x.pagerank, x.pageviews])).collectAsMap()
se.load_data(id_title_pagerank_pageviews) # this func is implemented in search frontend in order to set 
                                          # up an all the variables search functions might need.
start_search_colab = time.time()
#res = se.search_title('american')
# res2 = se.search_anchor()
res3 = se.search_body("art artist")
# res4 = se.get_pagerank()
# res5 = se.get_pageview()
#res6 = se.search('american')
end_search_colab = time.time()
print("search time in colab:")
print(end_search_colab - start_search_colab)      

spark.stop()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


filitered_query = ['art', 'artist']
candidates_tf_idf = {(12, 'art'): 0.0010400109671087217, (303, 'art'): 7.466312018106389e-05, (305, 'art'): 0.0002820839303453696, (307, 'art'): 7.621820915226756e-05, (308, 'art'): 0.00030109740005199317, (316, 'art'): 0.001966350367686486, (324, 'art'): 0.0008384179396958069, (339, 'art'): 0.0005833822733374528, (358, 'art'): 0.00018352603431740538, (569, 'art'): 0.00159582104323568, (573, 'art'): 0.0018491240901518896, (586, 'art'): 0.00017954226967120535, (593, 'art'): 0.00016790896910574629, (594, 'art'): 0.0007296684627241857, (597, 'art'): 0.0002643610564379438, (600, 'art'): 0.000473754925778159, (624, 'art'): 0.00017034861134271792, (628, 'art'): 0.0014720861102907688, (657, 'art'): 0.00013284942858179618, (676, 'art'): 0.0016577809262167092, (677, 'art'): 0.0005750482408612035, (690, 'art'): 0.00020950751315206926, (700, 'art'): 0.001082677032793852, (701, 'art'): 0.00010246065720418526, (717, 'art'): 9.704285646163028e-05, (736, 'art'): 0.

NameError: ignored

In [406]:
res6

[(316, 'Academy Award for Best Production Design'),
 (593, 'Animation'),
 (752, 'Art'),
 (780, 'Atlas (disambiguation)'),
 (983, 'Albert Camus'),
 (1141, 'Augustin-Jean Fresnel'),
 (1212, 'Artist'),
 (12, 'Anarchism'),
 (308, 'Aristotle'),
 (309, 'An American in Paris'),
 (339, 'Ayn Rand'),
 (590, 'Austin (disambiguation)'),
 (594, 'Apollo'),
 (628, 'Aldous Huxley'),
 (674, 'Anatomy'),
 (700, 'Arthur Schopenhauer'),
 (717, 'Alberta'),
 (737, 'Afghanistan'),
 (738, 'Albania'),
 (746, 'Azerbaijan'),
 (751, 'Aikido'),
 (791, 'Asteroid'),
 (800, 'Anime'),
 (808, 'Alfred Hitchcock'),
 (841, 'Attila'),
 (844, 'Amsterdam'),
 (864, 'Andy Warhol'),
 (872, 'Akira Kurosawa'),
 (880, 'ABBA'),
 (894, 'Agnosticism'),
 (922, 'Anxiety'),
 (924, 'A. A. Milne'),
 (931, 'The Amazing Spider-Man'),
 (954, 'Albert Speer'),
 (984, 'Agatha Christie'),
 (1008, 'April 6'),
 (1009, 'April 12'),
 (1010, 'April 15'),
 (1011, 'April 30'),
 (1012, 'August 22'),
 (1013, 'August 27'),
 (1016, 'Achill Island'),
 (1017,

In [ ]:
"""
def load_data():
    # load the three indexes
    idx_body = InvertedIndex.read_index('.', 'body_index')
    idx_title = InvertedIndex.read_index('.', 'title_index')
    idx_anchor = InvertedIndex.read_index('.', 'anchor_index')

    # declare the variables we will need in other functions (search) as global
    global body_words_pls
    global title_words_pls
    global anchor_words_pls
    global id_pagerank_title_df

    # make a dictionary such as {word1: pls1, word2: pls2 ...}
    body_words_pls = dict(zip(*zip(*idx_body.posting_lists_iter())))
    title_words_pls = dict(zip(*zip(*idx_title.posting_lists_iter())))
    anchor_words_pls = dict(zip(*zip(*idx_anchor.posting_lists_iter())))


    # load the csv that contains all of the pageranks, titles, and pageviews as a spark DF
    spark = SparkSession.builder.appName("Example").getOrCreate()
    id_pagerank_title_df = spark.read.csv("colab_pagerank.csv", header=True, inferSchema=True)
    spark.stop()
"""

In [ ]:
#global id_pagerank_title_df

# idx_body = InvertedIndex.read_index('.', 'body_index')
# idx_title = InvertedIndex.read_index('.', 'title_index')
# idx_anchor = InvertedIndex.read_index('.', 'anchor_index')

# id_pagerank_title_df = spark.read.csv("colab_pagerank.csv", header=True, inferSchema=True)
# id_pagerank_title_df = pd.read_csv("colab_pagerank.csv", index_col=0)

# body_words_pls = dict(zip(*zip(*idx_body.posting_lists_iter())))
# title_words_pls = dict(zip(*zip(*idx_title.posting_lists_iter())))
# anchor_words_pls = dict(zip(*zip(*idx_anchor.posting_lists_iter())))

In [ ]:
# uncomment the code below and execute to reload the module when you make 
# changes to search_frontend.py (after you upload again).


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<module 'colab_search_frontend' from '/content/drive/MyDrive/Information Retrieval/Project/information-retrieval-final-project/colab_cache/colab_search_frontend.py'>

In [ ]:
from flask_ngrok import run_with_ngrok
run_with_ngrok(se.app) 
se.app.run()

 * Serving Flask app "search_frontend" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://be08-35-196-42-90.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


# Testing your app

Once your app is running you can query it. You can simply do that by navigating to the URL that ngrok gave you above or through code in a different python session. For example, once the frontend app is running, you can navigate to:
http://YOUR_SERVER_DOMAIN/search?query=hello+world where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io, which is printed above in Colab or that is your external IP on GCP.

The code below shows how to issue a query from python. This is also how our testing code will issue queries to your search engine, so make sure to test your search engine this way after you deploy it to GCP and before submission. Command line instructions for deploying your search engine to GCP are available at `run_frontend_in_gcp.sh`. Note that we will not only issue training queries to your search engine, but also test queries, i.e. queries that you've never seen before.

In [ ]:
import json

with open('queries_train.json', 'rt') as f:
  queries = json.load(f)

In [ ]:
def average_precision(true_list, predicted_list, k=40):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    precisions = []
    for i,doc_id in enumerate(predicted_list):        
        if doc_id in true_set:
            prec = (len(precisions)+1) / (i+1)            
            precisions.append(prec)
    if len(precisions) == 0:
        return 0.0
    return round(sum(precisions)/len(precisions),3)

In [ ]:
import requests
from time import time
# url = 'http://35.232.59.3:8080'
# place the domain you got from ngrok or GCP IP below. 
url = 'http://XXXX-XX-XX-XX-XX.ngrok.io'

qs_res = []
for q, true_wids in queries.items():
  duration, ap = None, None
  t_start = time()
  try:
    res = requests.get(url + '/search', {'query': q}, timeout=35)
    duration = time() - t_start
    if res.status_code == 200:
      pred_wids, _ = zip(*res.json())
      ap = average_precision(true_wids, pred_wids)
  except:
    pass
  
  qs_res.append((q, duration, ap))

In [1]:
d = {1:'a', 2:'b'}
for i in d:
  print(i)
  

1
2
